In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions anda
# limitations under the License.
# ==============================================================================

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_models_entertainment-with-pretrained-embeddings/nvidia_logo.png" style="width: 90px; float: right;">

# Retrieval with hyperparameter optimization

## Overview

In this use case we will perform hyperparameter optimization using [optuna](https://optuna.org/). Optuna is an open source hyperparameter optimization framework which automates hyperparameter search and can be used across a wide set of scenarios.

We will look at optimizing candidate retrieval on a dataset from a Kaggle competition, the [H&M Personalized Fashion Recommendations challenge](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations).

Hyperparameter optimization can be arbitrarily complex -- in this use case, we will look at optimizing the learning rate and embedding dimensionality to achieve best results on candidate generation. We will train a Matrix Factorization model on user-item pairs and maximize the hit rate at 100 (how many out of top 100 retrieved candidates were among purchased items).

### Learning objectives

- How to run a hyperoptimization experiment
- Candidate generation using Merlin Models

## Downloading and preparing the dataset

Let's begin by downloading the dataset. Please find the data on Kaggle [here](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/data). We will only use `transactions_train.csv` which lists items that were purchased and maps the transactions to customers.

Please download the `transactions_train.csv` file and store it alongside the current notebook.

Let us read in the file and look at the data.

In [2]:
import cudf
import numpy as np

transactions = cudf.read_csv('transactions_train.csv', parse_dates=['t_dat'])
transactions.head()

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


Let's assign the last week to our validation set and treat the rest of that data as our train set. Additionally, let us remove purchases in our train set that were performed by customers who do not appear in our validation set.

In [3]:
seven_days_ago = transactions.t_dat.max() - np.timedelta64(7, 'D')

train_set = transactions[transactions.t_dat < seven_days_ago]
validation_set = transactions[transactions.t_dat >= seven_days_ago]

validation_set_customers = validation_set.customer_id.unique()
train_set = train_set[train_set.customer_id.isin(validation_set_customers)]

Before we can proceed with training we need to preprocess our data.

We will only use `customer_id` and `article_id` pairs. Still, a neural network expects them to be represented as continuous integers. We need to go from how they are represented in our dataset to that desired representation.

In order to do so, we will leverage `nvtabular` and the `Categorify` operator.

In [4]:
import nvtabular as nvt
from merlin.schema.tags import Tags
from merlin.models.tf.dataset import BatchedDataset

2022-10-04 07:47:00.216818: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-04 07:47:00.217347: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-04 07:47:00.217530: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-04 07:47:00.229052: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate 

We represent our data as `Merlin` `Datasets`.

In [5]:
train_set = nvt.Dataset(train_set)
validation_set = nvt.Dataset(validation_set)

We now define the operations we want to apply to our data.

In [6]:
customer_id = ['customer_id'] >> nvt.ops.Categorify() >> nvt.ops.AddMetadata(tags=[Tags.USER_ID])
article_id = ['article_id'] >> nvt.ops.Categorify() >> nvt.ops.AddMetadata(tags=[Tags.ITEM_ID])

And we proceed with fitting the workflow and transforming both the train and validation datasets.

In [7]:
workflow = nvt.Workflow(customer_id + article_id)
train_set_transformed = workflow.fit_transform(train_set)

validation_set_transformed = workflow.transform(validation_set)

/usr/local/lib/python3.8/dist-packages/cudf/core/frame.py:384: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


We are now ready to train our model and perform hyperparameter optimization.

## Hyperparameter optimization using optuna

We will train a retrieval model. Customers have performed a number of purchases in the last week of data that we are using as our validation set.

Our objective will be to train a retrieval model, Matrix Factorization, and to generate 100 candidates for each customer.

The goal is to maximize the count of purchases that appear among our candidates.

### The 3 components of hyperparemter optimization with optuna

With optuna, you create a `study`. A `study` is an optimization session with a number of trials. A `trial` is a single experiment, a call of the objective function.

A `parameter` is a variable whose value we will will optimize.

This is a brief primer but should provide you with all the information necessary to get started with optuna. You can find further information in optuna documentation [here](https://optuna.readthedocs.io/en/stable/tutorial/index.html).

Below we will run an optuna study. The hyperparameters we will optimize are:

* embedding dimensionality
* learning rate
* number of epochs

Embedding dimensionality is how we control the capacity of the model. We would like to provide it with just the right amount of expressive power. If we endow our model with too great of a capacity, our model will overfit to our training data and it's ability to generalize to unseen data will suffer.

On the other hand, if we train too simple of a model, it's performance will be limited as it will not be able to capture the signal in our train data.

The learning rate and number of epochs deal with the technicalities of training our model. As the model might train differently depending on it's capacity, we will perform a grid search across all these parameters in order to find a combination that will perform best.

### The setup of our study

Below we define an `objective` function. Running it is a single `trial` in our optimization `study`.

We `search_space` we define the values we would like our `study` to run `trials` with.

In [8]:
import merlin.models.tf as mm
import tensorflow as tf
import optuna

In [9]:
def calcualate_hit_rate_at_100(model):
    item_features = train_set_transformed.schema.select_by_tag(Tags.ITEM_ID).column_names
    item_dataset = train_set_transformed.to_ddf()[item_features].drop_duplicates().compute()
    
    item_dataset = nvt.Dataset(item_dataset)
    top_k_rec = model.to_top_k_recommender(item_dataset, 100)
    
    users_schema = train_set_transformed.schema.select_by_tag(Tags.USER_ID)
    user_features = users_schema.column_names

    unique_users = train_set_transformed.to_ddf()[user_features].drop_duplicates().compute()
    users_dataset = nvt.Dataset(unique_users, schema=users_schema)
    _, cls_idxs = top_k_rec.predict(BatchedDataset(users_dataset, 100, shuffle=False, schema=users_schema))
    
    customer_id_mapping = cudf.read_parquet('.//categories/unique.customer_id.parquet')
    customer_ids_preds = customer_id_mapping.to_pandas().customer_id.iloc[1:].values
    article_id_mapping = cudf.read_parquet('.//categories/unique.article_id.parquet')
    
    validation_set_df = validation_set.compute()
    validation_set_df = validation_set_df.drop_duplicates(['customer_id', 'article_id'])
    val_cust2purchases = validation_set_df.to_pandas().groupby('customer_id')['article_id'].apply(list)
    
    id2a = article_id_mapping.to_pandas().article_id.to_dict()
    
    hit_rate = 0
    for i in range(cls_idxs.shape[0]):
        current_cust = customer_ids_preds[i]
        purchases = set(val_cust2purchases[current_cust])
        candidates = set(int(id2a[c]) for c in cls_idxs[i])
        hit_rate += len(purchases.intersection(candidates))
        
    return hit_rate

In [10]:
def objective(trial):
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1)
    num_epochs = trial.suggest_int('num_epochs', 1, 100)
    embedding_dim = trial.suggest_int('embedding_dim', 4, 128)
    
    model = mm.MatrixFactorizationModel(train_set_transformed.schema, embedding_dim)

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, label_smoothing=0,
    )

    model.compile(optimizer, loss=loss)

    model.fit(train_set_transformed, validation_data=validation_set_transformed, batch_size=1024, epochs=num_epochs)
    
    return calcualate_hit_rate_at_100(model)

In [11]:
search_space = {
    'learning_rate': [5e-3, 1e-3, 1e-4],
    'num_epochs': [1, 3, 6, 10],
    'embedding_dim': [8, 16, 24, 32]
}

Our `study` will execute 48 trials (4 * 4 * 3) for us. I am capturing the output below as otherwise the notebook would be very hard to read.

We will use a `GridSampler` that will iterate over all the possible combinations of the hyperaparams we chose for our study. This is just one of the many ways supported by `Optuna` for exploring the hyperparam space.

We set the `n_trials` parameter to 100. Still, `Optuna` will only run 48 runs and will terminate when it has tested all the possible hyperaparameter combinations.

In [12]:
%%capture

study = optuna.create_study(sampler=optuna.samplers.GridSampler(search_space), direction='maximize')
study.optimize(objective, n_trials=100)

[I 2022-10-04 07:47:01,350] A new study created in memory with name: no-name-5bbf326d-1160-4509-9f31-e151b817967e


INFO:tensorflow:Assets written to: /tmp/tmp3qtqqwdb/assets


[I 2022-10-04 07:47:36,883] Trial 0 finished with value: 2710.0 and parameters: {'learning_rate': 0.005, 'num_epochs': 1, 'embedding_dim': 8}. Best is trial 0 with value: 2710.0.


INFO:tensorflow:Assets written to: /tmp/tmpf3l6ix__/assets


[I 2022-10-04 07:48:08,032] Trial 1 finished with value: 4459.0 and parameters: {'learning_rate': 0.001, 'num_epochs': 1, 'embedding_dim': 8}. Best is trial 1 with value: 4459.0.


...

[I 2022-10-04 09:22:56,566] Trial 45 finished with value: 3480.0 and parameters: {'learning_rate': 0.0001, 'num_epochs': 6, 'embedding_dim': 8}. Best is trial 31 with value: 6812.0.


INFO:tensorflow:Assets written to: /tmp/tmp_xwd6x73/assets


[I 2022-10-04 09:23:28,992] Trial 46 finished with value: 237.0 and parameters: {'learning_rate': 0.0001, 'num_epochs': 1, 'embedding_dim': 16}. Best is trial 31 with value: 6812.0.


INFO:tensorflow:Assets written to: /tmp/tmpfkile1af/assets


[I 2022-10-04 09:26:01,694] Trial 47 finished with value: 6448.0 and parameters: {'learning_rate': 0.001, 'num_epochs': 6, 'embedding_dim': 24}. Best is trial 31 with value: 6812.0.


Now that we have performed hyperparameter optimization (a `study` in `Optuna`'s parlance), let's explore the results.

We can query the `study` for the best parameters along with te best attained result.

In [13]:
study.best_params

{'learning_rate': 0.001, 'num_epochs': 6, 'embedding_dim': 32}

In [14]:
study.best_value

6812.0

We can also query the `study` for the number of `trials` that have been run.

In [18]:
len(study.trials)

48

And if we would like, we can also take a look at each individual run as follows.

In [19]:
study.trials[0]

FrozenTrial(number=0, values=[2710.0], datetime_start=datetime.datetime(2022, 10, 4, 7, 47, 1, 350748), datetime_complete=datetime.datetime(2022, 10, 4, 7, 47, 36, 883536), params={'learning_rate': 0.005, 'num_epochs': 1, 'embedding_dim': 8}, distributions={'learning_rate': FloatDistribution(high=0.1, log=False, low=1e-05, step=None), 'num_epochs': IntDistribution(high=100, log=False, low=1, step=1), 'embedding_dim': IntDistribution(high=128, log=False, low=4, step=1)}, user_attrs={}, system_attrs={'search_space': OrderedDict([('embedding_dim', [8, 16, 24, 32]), ('learning_rate', [0.005, 0.001, 0.0001]), ('num_epochs', [1, 3, 6, 10])]), 'grid_id': 0}, intermediate_values={}, trial_id=0, state=TrialState.COMPLETE, value=None)

In [20]:
study.trials[0].params

{'learning_rate': 0.005, 'num_epochs': 1, 'embedding_dim': 8}

## Summary